In [1]:
#export
"""
This module is for color formats, units and whatnot. This is exposed
automatically with::

   from k1lib.imports import *
   fmt.txt # exposed
"""
import k1lib, math, re; from k1lib import cli
from typing import Dict, Iterator, Tuple
pygments = k1lib.dep.pygments; bs4 = k1lib.dep.bs4
__all__ = ["generic", "metricPrefixes", "size", "fromSize", "scale", "fromScale", "dollar", "sizeOf",
           "comp", "compRate", "time", "item", "throughput", "txt",
           "js", "py", "html", "sql", "cpp", "java", "php", "ruby",
           "h", "pre", "row", "col", "colors", "rmAnsi"]

In [2]:
#export
k1lib.settings.add("fmt", k1lib.Settings().add("separator", True, "whether to have a space between the number and the unit"), "from k1lib.fmt module");
settings = k1lib.settings.fmt
metricPrefixes = {-8:"y",-7:"z",-6:"a",-5:"f",-4:"p",-3:"n",-2:"u",-1:"m",0:"",1:"k",2:"M",3:"G",4:"T",5:"P",6:"E",7:"Z",8:"Y"}
#metricPrefixes = ["", "k", "M", "G", "T", "P", "E", "Z", "Y"]
def generic(x, units:Dict[int, str]):
    c = " " if settings.separator else ""
    for i, unit in units.items():
        upperBound = 1000 * 1000**i
        if abs(x) < upperBound:
            return f"{round(1e3*x/upperBound, 2)}{c}{unit}"
    return (f"{round(1e3*x/upperBound, 2)}{c}{unit}").strip()
def _jsF_generic(units):
    def inner(meta):
        fIdx = cli.init._jsFAuto(); dataIdx = cli.init._jsDAuto(); unitsIdx = cli.init._jsDAuto()
        return f"""
{unitsIdx} = {[list(x) for x in units.items()]};
{fIdx} = ({dataIdx}) => {{
    for (const e of {unitsIdx}) {{
        const upperBound = 1000 * 1000**e[0];
        if (Math.abs({dataIdx}) < upperBound) return `${{Math.round(1e3*{dataIdx}/upperBound*100)/100}}{' ' if settings.separator else ''}${{e[1]}}`;
    }}
}}""", fIdx
    return inner

In [3]:
#export
sizes = {i: f"{p}B" for i, p in metricPrefixes.items() if i >= 0}; #sizes[0] = "bytes"
def size(_bytes=0):
    """Formats disk size.
Example::

    # returns "50.0 bytes"
    fmt.size(50)
    # returns "12.0 MB"
    fmt.size(1.2e7)
"""
    return generic(_bytes, sizes)
k1lib.settings.cli.kjs.jsF[size] = _jsF_generic(sizes)
sizeInv = {"k": 1e3, "m": 1e6, "g": 1e9, "t": 1e12, "p": 1e15, "e": 1e18, "z": 1e21, "y": 1e24}
def fromSize(s:str) -> int: # this is ugly I know, doesn't fit well into others. But making a generalized version seems hard, and I just need this right now
    """Grabs size from string representation.
Example::

    fmt.fromSize("31.5k") # returns 31500
    fmt.fromSize("31.5kB") # also returns 31500
"""
    s = s.lower().replace(" ", "").rstrip("b"); ch = s[-1]
    if ch in sizeInv: return int(float(s[:-1])*sizeInv[ch])
    return int(s)

In [4]:
assert size(50) == "50.0 B"
assert size(1.2e7) == "12.0 MB"
assert fromSize("31.5k") == 31500
assert fromSize("31.5kB") == 31500

In [57]:
#export
scalePostfixes = {0: '', 1: 'thousand', 2: 'million', 3: 'billion', 4: 'trillion', 5: 'quadrillion', 6: 'quintillion', 7: 'sextillion', 8: 'septillion', 9: 'octillion', 10: 'nonillion', 11: 'decillion', 12: 'undecillion', 13: 'duodecillion', 14: 'tredecillion', 15: 'quattuordecillion', 16: 'quindecillion', 17: 'sexdecillion', 18: 'septendecillion', 19: 'octodecillion', 20: 'novemdecillion', 21: 'vigintillion', 22: 'unvigintillion', 23: 'duovigintillion', 24: 'trevigintillion', 25: 'quattuorvigintillion', 26: 'quinvigintillion', 27: 'sexvigintillion', 28: 'septenvigintillion', 29: 'octovigintillion', 30: 'novemvigintillion', 31: 'trigintillion', 32: 'untrigintillion', 33: 'duotrigintillion'}
def scale(n:0):
    """Formats regular number with 'thousands', 'millions' and so on.
Example::

    fmt.scale(13_500) # returns '13.5 thousand'

This works from 10^0 to 10^99"""
    return generic(n, scalePostfixes)
scaleInv = {x:10**(x*3.0) for x in scalePostfixes.keys()}
def fromScale(s:str):
    """Grab number from string representation.
Example::

    fmt.fromScale("13.5 thousand") # returns 13500.0

This works from 10^0 to 10^99"""
    s = s.lower()
    for x,y in reversed(scalePostfixes.items()):
        if y in s: return float(s.replace(y, ""))*scaleInv[x]
    return float(s)

In [56]:
assert scale(13_500) == '13.5 thousand'
assert fromScale("13.5 thousand") == 13500.0

In [5]:
#export
dollars = {i: f"{p}$" for i, p in metricPrefixes.items() if i >= 0}; #sizes[0] = "bytes"
def dollar(_dollar=0):
    """Formats dollar.
Example::

    # returns '50.0 $'
    fmt.dollar(50)
    # returns '12.0 M$'
    fmt.dollar(1.2e7)

I know this format looks kinda terrible, instead of "million" or "billion"
written out loud, but that's long, so let's still do the short metric prefix"""
    return generic(_dollar, dollars)
k1lib.settings.cli.kjs.jsF[dollar] = _jsF_generic(dollars)

In [6]:
assert dollar(50) == '50.0 $'
assert dollar(1.2e7) == '12.0 M$'

In [7]:
#export
def sizeOf(l:Iterator[float]) -> Tuple[str, Iterator[float]]:
    """Figures out appropriate scale, scales back the Iterator, and return both.
Example::

    x = torch.abs(torch.randn(2)) * 1e4 + 1e5
    label, t = fmt.sizeOf(x) # label is "kB"
    (t | toTensor()).min() # min value should be close to 100"""
    l = list(l | cli.apply(lambda n: abs(n)))
    v = l | cli.toMax()
    v = math.log10(v) if v > 0 else -math.log10(-v)
    idx = math.floor(v/3)
    coef = 1.0/1000**idx
    return sizes[idx], l | cli.apply(lambda x: x * coef) | cli.deref()

In [9]:
import torch
x = torch.abs(torch.randn(2)) * 1e4 + 1e5
label, i = sizeOf(x); assert label == "kB"
assert 50 < (i | cli.toTensor()).min() < 150

In [11]:
#export
computations = {i: f"{p}FLOPs" for i, p in metricPrefixes.items() if i >= 0}
def comp(flop=0):
    """Formats computation amount.
Example::

    # returns "50.0 FLOPs"
    fmt.computation(50)
    # returns "50.0 MFLOPs"
    fmt.computation(5e7)
"""
    return generic(flop, computations)
k1lib.settings.cli.kjs.jsF[comp] = _jsF_generic(computations)

In [12]:
assert comp(50) == "50.0 FLOPs"
assert comp(5e7) == "50.0 MFLOPs"
assert comp(1e30) == "1000000.0 YFLOPs"

In [13]:
#export
computationRates = {i: f"{p}FLOPS" for i, p in metricPrefixes.items() if i >= 0}
def compRate(flops=0):
    """Formats computation rate.
Example::

    # returns "50.0 FLOPS"
    fmt.computationRate(50)
    # returns "50.0 MFLOPS"
    fmt.computationRate(5e7)
"""
    return generic(flops, computationRates)
k1lib.settings.cli.kjs.jsF[compRate] = _jsF_generic(computationRates)

In [14]:
assert compRate(50) == "50.0 FLOPS"
assert compRate(5e7) == "50.0 MFLOPS"
assert compRate(1e30) == "1000000.0 YFLOPS"

In [15]:
#export
times = {i:f"{p}s" for i, p in metricPrefixes.items() if i <= 0}
def time(seconds=0):
    """Formats small times.
Example::

    fmt.time(50) # returns "50.0 s"
    fmt.time(4000) # returns "4000.0 s"
    fmt.time(0.02) # returns "20.0 ms"
    fmt.time(1e-5) # returns "10.0 us"
"""
    return generic(seconds, times)
k1lib.settings.cli.kjs.jsF[time] = _jsF_generic(times)

In [16]:
assert time(50) == "50.0 s"
assert time(4000) == "4000.0 s"
assert time(0.02) == "20.0 ms"
assert time(1e-5) == "10.0 us"
assert time(1e-10) == "100.0 ps"
assert time(1e-25) == "0.1 ys"

In [17]:
#export
items = {0: "", 1: "k", 2: "M", 3: "B", 4: "T"}
def item(n=0):
    """Formats generic item.
Example::

    # returns "50.0"
    fmt.item(50)
    # returns "500.0 k"
    fmt.item(5e5)
"""
    return generic(n, items)
k1lib.settings.cli.kjs.jsF[item] = _jsF_generic(items)

In [18]:
assert item(50) == "50.0 "
assert item(5e5) == "500.0 k"

In [19]:
#export
def throughput(n, unit=""):
    """Formats item throughput.
Example::

    # returns "3.16/year"
    fmt.throughput(1e-7)
    # returns "2.63/month"
    fmt.throughput(1e-6)
    # returns "3.6/hour"
    fmt.throughput(1e-3)
    # returns "100.0 k/s"
    throughput(1e5)
    
    # returns "100.0 k epochs/s"
    throughput(1e5, " epochs")

:param n: items per second
:param unit: optional item unit"""
    if n < 10/(365.25*86400): return item(n*(365.25*86400)) + f"{unit}/year"
    if n < 10/(30.4375*86400): return item(n*(30.4375*86400)) + f"{unit}/month"
    if n < 10/86400: return item(n*86400) + f"{unit}/day"
    if n < 10/3600: return item(n*3600) + f"{unit}/hour"
    if n < 10/60: return item(n*60) + f"{unit}/minute"
    return item(n) + f"{unit}/s"

In [20]:
assert throughput(1e-7) == "3.16 /year"
assert throughput(1e-6) == "2.63 /month"
assert throughput(1e-3) == "3.6 /hour"
assert throughput(1e5) == "100.0 k/s"
assert throughput(1e5, " epochs") == "100.0 k epochs/s"

In [21]:
#export
_esc = '\033['
_end = f'{_esc}0m'
class txt:
    """Text formatting.
Example::

    # will print out red text
    print(fmt.txt.red("some text"))"""
    @staticmethod
    def darkcyan(s:str):  return f"{_esc}36m{s}{_end}"
    @staticmethod
    def red(s:str):       return f"{_esc}91m{s}{_end}"
    @staticmethod
    def green(s:str):     return f"{_esc}92m{s}{_end}"
    @staticmethod
    def yellow(s:str):    return f"{_esc}93m{s}{_end}"
    @staticmethod
    def blue(s:str):      return f"{_esc}94m{s}{_end}"
    @staticmethod
    def purple(s:str):    return f"{_esc}95m{s}{_end}"
    @staticmethod
    def cyan(s:str):      return f"{_esc}96m{s}{_end}"
    @staticmethod
    def bold(s:str):      return f"{_esc}1m{s}{_end}"
    @staticmethod
    def grey(s:str):      return f"{_esc}38;2;150;150;150m{s}{_end}"
    @staticmethod
    def darkgrey(s:str):  return f"{_esc}38;2;100;100;100m{s}{_end}"
    @staticmethod
    def underline(s:str): return f"{_esc}4m{s}{_end}"
    @staticmethod
    def identity(s:str):  return f"{s}"

In [34]:
#export
def pygmentsCss(): return "<style>" + pygments.formatters.HtmlFormatter().get_style_defs(".highlight") + "</style>"
def js(s:str) -> str:
    """Makes javascript code pretty, returns html"""
    return k1lib.viz.Html(pygmentsCss() + pygments.highlight(s, pygments.lexers.JavascriptLexer(), pygments.formatters.HtmlFormatter()))
def py(s:str) -> str:
    """Makes python code pretty, returns html"""
    return k1lib.viz.Html(pygmentsCss() + pygments.highlight(s, pygments.lexers.PythonLexer(), pygments.formatters.HtmlFormatter()))
def html(s:str) -> str:
    """Makes html code pretty, returns html"""
    s = bs4.BeautifulSoup(s, 'html.parser').prettify()
    return k1lib.viz.Html(pygmentsCss() + pygments.highlight(s, pygments.lexers.HtmlLexer(), pygments.formatters.HtmlFormatter()))
def sql(s:str) -> str:
    """Makes sql code pretty, returns html"""
    return k1lib.viz.Html(pygmentsCss() + pygments.highlight(s, pygments.lexers.SqlLexer(), pygments.formatters.HtmlFormatter()))
def cpp(s:str) -> str:
    """Makes cpp code pretty, returns html"""
    return k1lib.viz.Html(pygmentsCss() + pygments.highlight(s, pygments.lexers.CLexer(), pygments.formatters.HtmlFormatter()))
def java(s:str) -> str:
    """Makes java code pretty, returns html"""
    return k1lib.viz.Html(pygmentsCss() + pygments.highlight(s, pygments.lexers.JavaLexer(), pygments.formatters.HtmlFormatter()))
def php(s:str) -> str:
    """Makes php code pretty, returns html"""
    return k1lib.viz.Html(pygmentsCss() + pygments.highlight(s, pygments.lexers.PhpLexer(), pygments.formatters.HtmlFormatter()))
def ruby(s:str) -> str:
    """Makes ruby code pretty, returns html"""
    return k1lib.viz.Html(pygmentsCss() + pygments.highlight(s, pygments.lexers.RubyLexer(), pygments.formatters.HtmlFormatter()))

In [26]:
py("""
def f(x:int):
    return x + 3
""")

'<style>pre { line-height: 125%; }\ntd.linenos .normal { color: inherit; background-color: transparent; padding-left: 5px; padding-right: 5px; }\nspan.linenos { color: inherit; background-color: transparent; padding-left: 5px; padding-right: 5px; }\ntd.linenos .special { color: #000000; background-color: #ffffc0; padding-left: 5px; padding-right: 5px; }\nspan.linenos.special { color: #000000; background-color: #ffffc0; padding-left: 5px; padding-right: 5px; }\n.highlight .hll { background-color: #ffffcc }\n.highlight { background: #f8f8f8; }\n.highlight .c { color: #3D7B7B; font-style: italic } /* Comment */\n.highlight .err { border: 1px solid #FF0000 } /* Error */\n.highlight .k { color: #008000; font-weight: bold } /* Keyword */\n.highlight .o { color: #666666 } /* Operator */\n.highlight .ch { color: #3D7B7B; font-style: italic } /* Comment.Hashbang */\n.highlight .cm { color: #3D7B7B; font-style: italic } /* Comment.Multiline */\n.highlight .cp { color: #9C6500 } /* Comment.Preproc */\n.highlight .cpf { color: #3D7B7B; font-style: italic } /* Comment.PreprocFile */\n.highlight .c1 { color: #3D7B7B; font-style: italic } /* Comment.Single */\n.highlight .cs { color: #3D7B7B; font-style: italic } /* Comment.Special */\n.highlight .gd { color: #A00000 } /* Generic.Deleted */\n.highlight .ge { font-style: italic } /* Generic.Emph */\n.highlight .gr { color: #E40000 } /* Generic.Error */\n.highlight .gh { color: #000080; font-weight: bold } /* Generic.Heading */\n.highlight .gi { color: #008400 } /* Generic.Inserted */\n.highlight .go { color: #717171 } /* Generic.Output */\n.highlight .gp { color: #000080; font-weight: bold } /* Generic.Prompt */\n.highlight .gs { font-weight: bold } /* Generic.Strong */\n.highlight .gu { color: #800080; font-weight: bold } /* Generic.Subheading */\n.highlight .gt { color: #0044DD } /* Generic.Traceback */\n.highlight .kc { color: #008000; font-weight: bold } /* Keyword.Constant */\n.highlight .kd { color: #008000; font-weight: bold } /* Keyword.Declaration */\n.highlight .kn { color: #008000; font-weight: bold } /* Keyword.Namespace */\n.highlight .kp { color: #008000 } /* Keyword.Pseudo */\n.highlight .kr { color: #008000; font-weight: bold } /* Keyword.Reserved */\n.highlight .kt { color: #B00040 } /* Keyword.Type */\n.highlight .m { color: #666666 } /* Literal.Number */\n.highlight .s { color: #BA2121 } /* Literal.String */\n.highlight .na { color: #687822 } /* Name.Attribute */\n.highlight .nb { color: #008000 } /* Name.Builtin */\n.highlight .nc { color: #0000FF; font-weight: bold } /* Name.Class */\n.highlight .no { color: #880000 } /* Name.Constant */\n.highlight .nd { color: #AA22FF } /* Name.Decorator */\n.highlight .ni { color: #717171; font-weight: bold } /* Name.Entity */\n.highlight .ne { color: #CB3F38; font-weight: bold } /* Name.Exception */\n.highlight .nf { color: #0000FF } /* Name.Function */\n.highlight .nl { color: #767600 } /* Name.Label */\n.highlight .nn { color: #0000FF; font-weight: bold } /* Name.Namespace */\n.highlight .nt { color: #008000; font-weight: bold } /* Name.Tag */\n.highlight .nv { color: #19177C } /* Name.Variable */\n.highlight .ow { color: #AA22FF; font-weight: bold } /* Operator.Word */\n.highlight .w { color: #bbbbbb } /* Text.Whitespace */\n.highlight .mb { color: #666666 } /* Literal.Number.Bin */\n.highlight .mf { color: #666666 } /* Literal.Number.Float */\n.highlight .mh { color: #666666 } /* Literal.Number.Hex */\n.highlight .mi { color: #666666 } /* Literal.Number.Integer */\n.highlight .mo { color: #666666 } /* Literal.Number.Oct */\n.highlight .sa { color: #BA2121 } /* Literal.String.Affix */\n.highlight .sb { color: #BA2121 } /* Literal.String.Backtick */\n.highlight .sc { color: #BA2121 } /* Literal.String.Char */\n.highlight .dl { color: #BA2121 } /* Literal.String.Delimiter */\n.highlight .sd { color: #BA2121; font-style: italic } /* Literal.String.Doc */\n.highlight .s2 { color: #BA2121 } /* Literal.String.Double */\n.highlight .se { 

In [27]:
js("""
function f(x) {
    return x+4;
}
""")

'<style>pre { line-height: 125%; }\ntd.linenos .normal { color: inherit; background-color: transparent; padding-left: 5px; padding-right: 5px; }\nspan.linenos { color: inherit; background-color: transparent; padding-left: 5px; padding-right: 5px; }\ntd.linenos .special { color: #000000; background-color: #ffffc0; padding-left: 5px; padding-right: 5px; }\nspan.linenos.special { color: #000000; background-color: #ffffc0; padding-left: 5px; padding-right: 5px; }\n.highlight .hll { background-color: #ffffcc }\n.highlight { background: #f8f8f8; }\n.highlight .c { color: #3D7B7B; font-style: italic } /* Comment */\n.highlight .err { border: 1px solid #FF0000 } /* Error */\n.highlight .k { color: #008000; font-weight: bold } /* Keyword */\n.highlight .o { color: #666666 } /* Operator */\n.highlight .ch { color: #3D7B7B; font-style: italic } /* Comment.Hashbang */\n.highlight .cm { color: #3D7B7B; font-style: italic } /* Comment.Multiline */\n.highlight .cp { color: #9C6500 } /* Comment.Preproc */\n.highlight .cpf { color: #3D7B7B; font-style: italic } /* Comment.PreprocFile */\n.highlight .c1 { color: #3D7B7B; font-style: italic } /* Comment.Single */\n.highlight .cs { color: #3D7B7B; font-style: italic } /* Comment.Special */\n.highlight .gd { color: #A00000 } /* Generic.Deleted */\n.highlight .ge { font-style: italic } /* Generic.Emph */\n.highlight .gr { color: #E40000 } /* Generic.Error */\n.highlight .gh { color: #000080; font-weight: bold } /* Generic.Heading */\n.highlight .gi { color: #008400 } /* Generic.Inserted */\n.highlight .go { color: #717171 } /* Generic.Output */\n.highlight .gp { color: #000080; font-weight: bold } /* Generic.Prompt */\n.highlight .gs { font-weight: bold } /* Generic.Strong */\n.highlight .gu { color: #800080; font-weight: bold } /* Generic.Subheading */\n.highlight .gt { color: #0044DD } /* Generic.Traceback */\n.highlight .kc { color: #008000; font-weight: bold } /* Keyword.Constant */\n.highlight .kd { color: #008000; font-weight: bold } /* Keyword.Declaration */\n.highlight .kn { color: #008000; font-weight: bold } /* Keyword.Namespace */\n.highlight .kp { color: #008000 } /* Keyword.Pseudo */\n.highlight .kr { color: #008000; font-weight: bold } /* Keyword.Reserved */\n.highlight .kt { color: #B00040 } /* Keyword.Type */\n.highlight .m { color: #666666 } /* Literal.Number */\n.highlight .s { color: #BA2121 } /* Literal.String */\n.highlight .na { color: #687822 } /* Name.Attribute */\n.highlight .nb { color: #008000 } /* Name.Builtin */\n.highlight .nc { color: #0000FF; font-weight: bold } /* Name.Class */\n.highlight .no { color: #880000 } /* Name.Constant */\n.highlight .nd { color: #AA22FF } /* Name.Decorator */\n.highlight .ni { color: #717171; font-weight: bold } /* Name.Entity */\n.highlight .ne { color: #CB3F38; font-weight: bold } /* Name.Exception */\n.highlight .nf { color: #0000FF } /* Name.Function */\n.highlight .nl { color: #767600 } /* Name.Label */\n.highlight .nn { color: #0000FF; font-weight: bold } /* Name.Namespace */\n.highlight .nt { color: #008000; font-weight: bold } /* Name.Tag */\n.highlight .nv { color: #19177C } /* Name.Variable */\n.highlight .ow { color: #AA22FF; font-weight: bold } /* Operator.Word */\n.highlight .w { color: #bbbbbb } /* Text.Whitespace */\n.highlight .mb { color: #666666 } /* Literal.Number.Bin */\n.highlight .mf { color: #666666 } /* Literal.Number.Float */\n.highlight .mh { color: #666666 } /* Literal.Number.Hex */\n.highlight .mi { color: #666666 } /* Literal.Number.Integer */\n.highlight .mo { color: #666666 } /* Literal.Number.Oct */\n.highlight .sa { color: #BA2121 } /* Literal.String.Affix */\n.highlight .sb { color: #BA2121 } /* Literal.String.Backtick */\n.highlight .sc { color: #BA2121 } /* Literal.String.Char */\n.highlight .dl { color: #BA2121 } /* Literal.String.Delimiter */\n.highlight .sd { color: #BA2121; font-style: italic } /* Literal.String.Doc */\n.highlight .s2 { color: #BA2121 } /* Literal.String.Double */\n.highlight .se { 

In [23]:
k1lib.settings.cli.kjs.jsF

{str: <function k1lib.cli.kjs.<lambda>(meta)>,
 int: <function k1lib.cli.kjs.<lambda>(meta)>,
 float: <function k1lib.cli.kjs.<lambda>(meta)>,
 set: <function k1lib.cli.kjs.<lambda>(meta)>,
 list: <function k1lib.cli.kjs.<lambda>(meta)>,
 tuple: <function k1lib.cli.kjs.<lambda>(meta)>,
 <function abs(x, /)>: <function k1lib.cli.kjs.<lambda>(meta)>,
 <function json.loads(s, *, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)>: <function k1lib.cli.kjs.<lambda>(meta)>,
 <function json.dumps(obj, *, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, cls=None, indent=None, separators=None, default=None, sort_keys=False, **kw)>: <function k1lib.cli.kjs.<lambda>(meta)>,
 <function base64.b64decode(s, altchars=None, validate=False)>: <function k1lib.cli.kjs.<lambda>(meta)>,
 <function base64.b64encode(s, altchars=None)>: <function k1lib.cli.kjs.<lambda>(meta)>,
 <function round(number, ndigits=None)>: <functio

In [24]:
#export
def h(code:str, level:int=1) -> str:
    """Wraps content inside a 'h' html tag.
Example::

    fmt.h("abc", 2) # returns "<h2>abc</h2>"

:param level: what's the header level?"""
    return f"<h{level}>{code}</h{level}>"
def _jsF_h(meta, level=3):
    fIdx = cli.init._jsFAuto(); dataIdx = cli.init._jsDAuto()
    return f"{fIdx} = ({dataIdx}) => `<h{level}>${{{dataIdx}}}</h{level}>`", fIdx
k1lib.settings.cli.kjs.jsF[h] = _jsF_h
def pre(code:str, extras:str="") -> str:
    """Wraps content inside a 'pre' html tag.
Example::

    fmt.pre("abc")
"""
    return f"<pre style='font-family: monospace' {extras} >{code}</pre>"
def _jsF_pre(meta):
    fIdx = cli.init._jsFAuto(); dataIdx = cli.init._jsDAuto()
    return f"{fIdx} = ({dataIdx}) => `<pre style='font-family: monospace'>${{{dataIdx}}}</pre>`", fIdx
k1lib.settings.cli.kjs.jsF[pre] = _jsF_pre
def col(args, margin=10):
    """Creates a html col of all the elements.
Example::

    fmt.col(["abc", "def"]) | aS(IPython.display.HTML)
"""
    return args | cli.apply(lambda x: f"<div style='margin: {margin}px'>{x}</div>") | cli.join("") | cli.aS(lambda x: f"<div style='display: flex; flex-direction: column'>{x}</div>")
def _jsF_col(meta, margin=10):
    fIdx = cli.init._jsFAuto(); dataIdx = cli.init._jsDAuto()
    fIdx2 = cli.init._jsFAuto(); dataIdx2 = cli.init._jsDAuto()
    return f"""
{fIdx2} = ({dataIdx2}) => `<div style='margin: {margin}px'>${{{dataIdx2}}}</div>`
{fIdx}  = ({dataIdx})  => `<div style='display: flex; flex-direction: column'>${{{dataIdx}.map({fIdx2}).join('')}}</div>`
    """, fIdx
k1lib.settings.cli.kjs.jsF[col] = _jsF_col
def row(args, margin=10):
    """Creates a html row of all the elements.
Example::

    fmt.row(["abc", "def"]) | aS(IPython.display.HTML)
"""
    return args | cli.apply(lambda x: f"<div style='margin: {margin}px'>{x}</div>") | cli.join("") | cli.aS(lambda x: f"<div style='display: flex; flex-direction: row'>{x}</div>")
def _jsF_row(meta, margin=10):
    fIdx = cli.init._jsFAuto(); dataIdx = cli.init._jsDAuto()
    fIdx2 = cli.init._jsFAuto(); dataIdx2 = cli.init._jsDAuto()
    return f"""
{fIdx2} = ({dataIdx2}) => `<div style='margin: {margin}px'>${{{dataIdx2}}}</div>`
{fIdx}  = ({dataIdx})  => `<div style='display: flex; flex-direction: row'>${{{dataIdx}.map({fIdx2}).join('')}}</div>`
    """, fIdx
k1lib.settings.cli.kjs.jsF[row] = _jsF_row

In [25]:
row(["abc", "def"]) | cli.aS(IPython.display.HTML)

In [26]:
["abc", "def"] | (cli.toJsFunc() | cli.aS(row)) | cli.op().interface()

In [27]:
col(["abc", "def"]) | cli.aS(IPython.display.HTML)

In [28]:
["abc", "def"] | (cli.toJsFunc() | cli.aS(col)) | cli.op().interface()

In [29]:
#export
settings.add("colors", ["#8dd3c7", "#ffffb3", "#bebada", "#fb8072", "#80b1d3", "#fdb462", "#b3de69", "#fccde5", "#d9d9d9", "#bc80bd", "#ccebc5", "#ffed6f"], "List of colors to cycle through in fmt.colors()")
def colors():
    """Returns an infinite iterator that cycles through 12 colors.
Example::

    fmt.colors() | head(3) | deref()

Color scheme taken from https://colorbrewer2.org/#type=qualitative&scheme=Set3&n=12"""
    return settings.colors | cli.repeatFrom()

In [30]:
#export
ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
def rmAnsi(text):
    """Removes ansi escape characters, courtesy of https://stackoverflow.com/questions/14693701/how-can-i-remove-the-ansi-escape-sequences-from-a-string-in-python."""
    return ansi_escape.sub('', text)

In [31]:
assert rmAnsi('ls\r\n\x1b[00m\x1b[01;31mexamplefile.zip\x1b[00m\r\n\x1b[01;31m') == 'ls\r\nexamplefile.zip\r\n'

In [87]:
!../export.py fmt --upload=True

2024-02-22 14:01:38,084	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2024-02-22 14:01:38,091	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
14998   0   61%   
9600    1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.5.2
Uninstalling k1lib-1.5.2:
  Successfully uninstalled k1lib-1.5.2
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdi

In [60]:
!../export.py fmt

2024-03-20 04:06:11,627	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-03-20 04:06:11,637	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
15766   0   61%   
10122   1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.7
Uninstalling k1lib-1.7:
  Successfully uninstalled k1lib-1.7
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg

In [49]:
!../export.py fmt --bootstrap=True

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-12-31 22:15:31,546	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2023-12-31 22:15:31,556	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- bootstrapping
Current dir: /home/kelvin/repos/labs/k1lib, /home/kelvin/repos/labs/k1lib/k1lib/../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4.4.5
Uninstalling k1lib-1.4.4.5:
  Successfully uninstalled k1lib-1.4.4.5
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based to